In [ ]:
#!/usr/bin/env python

import numpy as np
import pandas as pd
from matplotlib import cm
from matplotlib.colors import to_hex
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf

import os
import constants as cs

In [ ]:
# Utility functions


#
# plot a time series data with monthly markers to determine seasonal trends
#
# Args:
#    dates  (type): timestamps of data collection
#    values (type): data values
#    xlabel (str) : plot x axis label
#    ylabel (str) : plot y axis label
#    title  (str) : plot title
#
# Returns:
#    None
#
def plot_monthly(
    dates: pd.DataFrame, values: pd.DataFrame, xlabel: str, ylabel: str, title: str
) -> None:
    # plot time series data
    plt.figure(figsize=(12, 4))
    plt.plot(
        dates,
        values,
        marker="o",
    )
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.title(title)

    colormap = cm.get_cmap("hsv")
    # plot monthly markers
    # monthly markers more clearly show the seasonality of the data
    for d, v in zip(dates, values):
        marker = r"$\rm{" + d.strftime("%b") + "}$"
        color = to_hex(colormap((d.month - 1)/12))
        plt.plot(d, v, marker=marker, markersize=12, color=color)
    plt.show()


#
# plot time series data
#
# Args:
#    dates  (type): timestamps of data collection
#    values (type): data values
#    xlabel (str) : plot x axis label
#    ylabel (str) : plot y axis label
#    title  (str) : plot title
#
# Returns:
#    None
#
def plot_ts(
    dates: pd.DataFrame, values: pd.DataFrame, xlabel: str, ylabel: str, title: str
) -> None:

    plt.figure(figsize=(12, 4))
    plt.plot(
        dates,
        values,
        marker="o",
    )
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.title(title)
    plt.show()

In [ ]:
# problem 3.4
# read the hours dataset into memory
hours = pd.read_csv(os.path.join(cs.DATASETS, "hours.dat"))
# create timestamps for the data
hours.set_index(pd.date_range("1982-07", periods=len(hours.index), freq="M"), inplace=True)

In [ ]:
# 3.4 a
plot_ts(hours.index, hours["hours"], "Time (Months)", "Hours", "Plot of Hours Dataset")

In [ ]:
# 3.4 b
plot_monthly(
    hours.index, hours["hours"], "Time (Months)", "Hours", "Plot of Hours Dataset"
)

In [ ]:
# problem 3.6
# read dataset and set timestamp
beersales = pd.read_csv(os.path.join(cs.DATASETS, "beersales.dat"))
beersales.set_index(
    pd.date_range("1975-01", periods=len(beersales.index), freq="M"), inplace=True
)
# not sure what this is calculated, but is necessary for part e
beersales["t"] = (beersales.index.year + beersales.index.month - 1) / 12

In [ ]:
# 3.6 a
plot_ts(
    beersales.index,
    beersales["beersales"],
    "Time (Months)",
    "Beer Sales (Millions of Barrels)",
    "Millions of Barrels sold in the U.S. from 1975 - 1990",
)

In [ ]:
# 3.6 b
plot_monthly(
    beersales.index,
    beersales["beersales"],
    "Time (Months)",
    "Beer Sales (Millions of Barrels)",
    "Millions of Barrels sold in the U.S. from 1975 - 1990",
)

In [ ]:
# 3.6 c
beersales["month"] = beersales.index.month
model = smf.ols("beersales ~ C(month)", data=beersales).fit()
model.summary()

In [ ]:
# 3.6 d
plot_monthly(beersales.index, model.resid_pearson, "Time (Months)", "Residual", "Residual over time")

In [ ]:
# 3.6 e
model = smf.ols("beersales ~ t + np.power(t, 2) + C(month)", data=beersales).fit()
model.summary()


In [ ]:
# 3.6 f
plot_monthly(beersales.index, model.resid_pearson, "Time (Months)", "Residual", "Residual over time")

In [ ]:
# 3.10 a

hours["t"] = (hours.index.year + hours.index.month - 1) / 12
model = smf.ols("hours ~ t + np.power(t, 2)", hours).fit()
model.summary()

In [ ]:
plot_monthly(hours.index, model.resid_pearson, "Time (Months)", "Residuals", "Residuals over time")

In [ ]:
from statsmodels.sandbox.stats.runs import runstest_1samp

z_stat, p_value = runstest_1samp(model.resid_pearson, cutoff=0, correction=False)
print('Z statistic:\t %.3f' % z_stat)
print('p-value:\t %e' % p_value)

In [ ]:
sm.graphics.tsa.plot_acf(model.resid_pearson, lags=20)

In [ ]:
plt.figure(figsize=(12, 8))
plt.hist(model.resid_pearson, edgecolor='black', facecolor='lightgray')
plt.xlabel('Residual')
plt.ylabel('Frequency')
plt.title('Histogram of Residuals')
plt.show()

import matplotlib as mpl

with mpl.rc_context():
    mpl.rc("figure", figsize=(12, 8))
    sm.ProbPlot(model.resid_pearson).ppplot(line='45')

plt.show()